# 5. QALI-specific querying

A real QALI query, used to to drive the search on the addresses listing page:


```sparql
PREFIX cn: <https://linked.data.gov.au/def/cn/>
PREFIX  rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  text: <http://jena.apache.org/text#>
PREFIX  addr: <https://w3id.org/profile/anz-address/>
PREFIX  prov: <http://www.w3.org/ns/prov#>
PREFIX  sdo:  <https://schema.org/>

SELECT *
FROM <urn:ladb:graph:addresses>
WHERE
{
  # Filter by thoroughfare type (street or water)
  VALUES ?resourceType {
    UNDEF 
  }

  {
    SELECT DISTINCT ?iri
    WHERE {
      {
        SELECT DISTINCT ?labelIri
        WHERE {
          # User text query
          ( ?labelIri ?score ?matchedLabel ?graph ) text:query "1 william st brisbane*"
        }
        ORDER BY DESC(?score)
      }

      # Filter by thoroughfare type (street or water)
      VALUES ?resourceType { UNDEF }

      ?labelIri  rdfs:label         ?label ;
                 sdo:additionalType  ?resourceType ;
                 rdfs:label          ?label ;
                 cn:isNameFor ?iri 
    }
    # Paginate
    OFFSET  0
    LIMIT   5
  }

  ?iri      cn:hasName            ?labelIri .
  ?labelIri  rdfs:label         ?label ;
             sdo:additionalType  ?resourceType

  OPTIONAL {
    ?iri sdo:identifier ?lot
    FILTER(DATATYPE(?lot) = <https://linked.data.gov.au/dataset/qld-addr/datatype/lot>)
  }
  OPTIONAL {
    ?iri sdo:identifier ?plan
    FILTER(DATATYPE(?plan) = <https://linked.data.gov.au/dataset/qld-addr/datatype/plan>)
  }
}
```

Try it at <https://training.cam.kurrawong.ai>...